In [ ]:
import pandas as pd
import os
from datetime import datetime
try:
    from utils.constants import categories
except ModuleNotFoundError:
    from notebooks.utils.constants import categories


In [17]:
try:
    %run 01_extract.ipynb
except Exception:
    %run notebooks/01_extract.ipynb

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14966 entries, 0 to 14965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Categoria          14966 non-null  object 
 1   Série              14966 non-null  object 
 2   Data de Início     14966 non-null  object 
 3   Hora de Início     14966 non-null  object 
 4   Data Final         14966 non-null  object 
 5   Hora de Término    14966 non-null  object 
 6   Valor              14966 non-null  float64
 7   Unidade de Medida  14966 non-null  object 
dtypes: float64(1), object(7)
memory usage: 935.5+ KB


In [12]:
dfs = extract_data() # type: ignore

# Funções Úteis

## Formatador

In [13]:
def dataset_formatter(df: pd.DataFrame):
  df = df.drop([
                'Hora de Início', 'Data Final', 'Hora de Término'], axis=1)

  df2 = pd.DataFrame(columns=categories)
  data = {category: 0 for category in categories}
  row = 0

  for date in df['Data de Início'].unique():
    df_by_date = df.loc[df['Data de Início'] == date]

    date = translate_date_to_en_US(date)
    for index, value in df_by_date.iterrows():
      category = value['Categoria'] + " " + value['Série']
      data[category] = value['Valor']

    df2.loc[row, 'Data de leitura'] = datetime.strptime(
     date, '%d %b %Y'
    )
    for key, value in data.items():
      df2.loc[row, key] = value

    # Velocidade média
    df2.loc[row, 'Velocidade Média Avanço'] = calculate_average_speed(
        df=df2,
        row=row,
        distance_col='Distância Percorrida em Avanço e em Marcha à Ré Avanço',
        time_col='Tempo em Avanço e em Marcha à Ré Avanço'
    )
    df2.loc[row, 'Velocidade Média Marcha à ré'] = calculate_average_speed(
        df=df2,
        row=row,
        distance_col='Distância Percorrida em Avanço e em Marcha à Ré Marcha à Ré',
        time_col='Tempo em Avanço e em Marcha à Ré Marcha à Ré'
    )

    # Horas de operação (aproximado)
    cols = ['Uso do Motor Marcha Lenta', 'Uso do Motor Carga Alta',
            'Uso do Motor Carga Média', 'Uso do Motor Carga Baixa']

    df2.loc[row, 'Horas de Operação'] = df2.loc[row, cols].sum()


    data.update(dict.fromkeys(data, 0)) # Evita a falta de dados
    row += 1

  # Convertendo objects para números
  df2[df2.columns.difference(['Data de leitura'])] = df2[
      df2.columns.difference(['Data de leitura'])].apply(pd.to_numeric)

  df2.sort_values(by=['Data de leitura'], inplace=True, ignore_index=True)

  return df2


def calculate_average_speed(**kwargs):
  df = kwargs.get('df')
  row = kwargs.get('row')
  distance_col = kwargs.get('distance_col')
  time_col = kwargs.get('time_col')

  try:
    distance = df.loc[row, distance_col]
    time = df.loc[row, time_col]
  except:
    distance = 0
    time = 0

  return distance/time if time != 0 else 0

def translate_date_to_en_US(date):
  date_slice = date.split(' ')

  day = date_slice[0]
  month = date_slice[1]
  year = date_slice[2]

  match month:
    case 'fev':
      return f'{day} Feb {year}'
    case 'abr':
      return f'{day} Apr {year}'
    case 'mai':
      return f'{day} May {year}'
    case 'ago':
      return f'{day} Aug {year}'
    case 'set':
      return f'{day} Sep {year}'
    case 'out':
      return f'{day} Oct {year}'
    case 'dez':
      return f'{day} Dec {year}'
    case _:
      return date

## Filtragem por horímetro

In [14]:
def filter_dataframe_by_engine_hour(dataframe, hour_meter):
  filtered_df = dataframe.copy()
  filtered_df['Horas do Motor'] = filtered_df['Horas de Operação'].cumsum() # horas do motor aproximado
  filtered_df = filtered_df[filtered_df['Horas do Motor'] <= hour_meter]
  return filtered_df

# Transformando os datasets

## Criando Pastas

In [15]:
os.makedirs("../data/processed/filtered/", exist_ok=True)
os.makedirs("../data/processed/full/", exist_ok=True)

## Formatando os datasets

In [16]:
for index, [name, df] in enumerate(dfs.items()):
    f_df = dataset_formatter(df)
    if index == 0:
        print(f_df.info())
        
    f_df.to_csv(f"../data/processed/full/df_{name}.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 32 columns):
 #   Column                                                            Non-Null Count  Dtype         
---  ------                                                            --------------  -----         
 0   Temperatura Máxima do Óleo Hidráulico Temperatura                 572 non-null    float64       
 1   Temp. Máx. do Óleo da Transm. Temperatura do Óleo da Transmissão  572 non-null    float64       
 2   Ajuste da Rotação da Transmissão 0,5 a 0,9                        572 non-null    float64       
 3   Ajuste da Rotação da Transmissão 1,0 a 1,4                        572 non-null    float64       
 4   Ajuste da Rotação da Transmissão 2,5 a 2,9                        572 non-null    float64       
 5   Ajuste da Rotação da Transmissão 3,0 a 3,5                        572 non-null    float64       
 6   Ajuste da Rotação da Transmissão 1,5 a 1,9                        572 non-

KeyboardInterrupt: 

## Filtrando por horímetro

In [ ]:
hour_meter = 1100

def full_dataframe(df: str):
    return pd.read_csv(f"../data/processed/full/{df}")

for index, dataframe in enumerate(os.listdir("../data/processed/full/")):
    df = full_dataframe(dataframe)
    f_df = filter_dataframe_by_engine_hour(df, hour_meter)

    if index == 0:
        print(f_df.info())

    f_df_name = dataframe.split(".")[0]  # removendo o .csv do final
    f_df.to_csv(f"../data/processed/filtered/{f_df_name}_{hour_meter}.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 91 entries, 0 to 90
Data columns (total 33 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Temperatura Máxima do Óleo Hidráulico Temperatura                 91 non-null     float64
 1   Temp. Máx. do Óleo da Transm. Temperatura do Óleo da Transmissão  91 non-null     float64
 2   Ajuste da Rotação da Transmissão 0,5 a 0,9                        91 non-null     float64
 3   Ajuste da Rotação da Transmissão 1,0 a 1,4                        91 non-null     float64
 4   Ajuste da Rotação da Transmissão 2,5 a 2,9                        91 non-null     float64
 5   Ajuste da Rotação da Transmissão 3,0 a 3,5                        91 non-null     float64
 6   Ajuste da Rotação da Transmissão 1,5 a 1,9                        91 non-null     float64
 7   Ajuste da Rotação da Transmissão 2,0 a 2,4